# SISPS-EXP-AST9AH-EIS

Main notebook for visual analysis of SISPS-EXP-SOC experiment EIS spectra and related properties.

[Lawrence Stanton](mailto:stnlaw003@myuct.ac.za)

- _Began_ March 2024
- _Revised_ December 2024


## Environment

The required environment is described in the '[environment.yml](environment.yml)' file intended for use with Anaconda. To create this environment, run the following command (or equivalently via your IDE):

```bash
conda env create -f environment.yml
```

```{note}
Last tested using Python 3.11
```


In [27]:
import time

# Begin timer
startTime = time.time()

In [28]:
import matplotlib
from  matplotlib import pyplot as plt
import darkdetect

# Modify default plotting style
if darkdetect.isDark():
	plt.style.use('dark_background')

plt.rcParams.update({
	'figure.dpi': 300,
	'figure.figsize': (8, 5),
})

%config InlineBackend.figure_formats = ['svg']

nyquistRealAxisLabel = r'Real Impedance (mΩ)'
nyquistImagAxisLabel = r'Imaginary Impedance (mΩ)'


## Electrochemical Impedance Spectroscopy (EIS)

### Import

Importing raw data from the EIS experiment, and converting it into a format that can be used for analysis.

All EIS spectra are plotted individually. Only a subset however exist as final measurements (the rest being discarded).

In [29]:
from Lib.eisImport import findMainEisFiles, findAllEisFiles, readEisFiles

eisDirectory = "../Data/EIS"

eisFiles = findMainEisFiles(eisDirectory)
eisRaw = readEisFiles(tuple(eisFiles))

eisAllFiles = findAllEisFiles(eisDirectory)
eisRawAll = readEisFiles(tuple(eisAllFiles))

Also import manual measurements and observations, importantly including temperature measurements.

In [30]:
from Lib.eisImport import (
    readMeasurementsAndObservations,
    groupMeasurementsAndObservations,
    inferObservationTestNames,
)

observationsFile = "../Data/UCT AST9AH Measurements and Observations.xlsx"
eisObservations = groupMeasurementsAndObservations(
    readMeasurementsAndObservations(observationsFile)
)
inferObservationTestNames(eisObservations)

Plots for visual check on temperature ranges.

In [31]:
from Lib.eisPlot import plotEisTestTemperatureRanges

temperatureRangeFig = plotEisTestTemperatureRanges(
    eisObservations,
    "../Plots/Temperature",
    "EIS_Temperature_Ranges.svg",
)

### Raw EIS Plots

Plots all EIS spectra individually. This is useful for visual inspection of the raw data.

Plots are saved under the `Plots` directory and are only generated once based on the existence of the target directories (which are not committed to the repository ala `.gitignore`). To regenerate the plots, delete the `Plots` directory and rerun this cell.

Overlaid plot of all tests is also generated for a quick visual comparison of all tests and reference against preprocessing.

In [32]:
from Lib.eisPlot import plotNyquist, plotIndividualNyquist

plotIndividualNyquist(
    eisRaw,
    saveDir="../Plots/EIS/Raw/Main",
    scatter=False,
)
plotIndividualNyquist(
    eisRawAll,
    saveDir="../Plots/EIS/Raw/All",
    scatter=False,
)

plt.close("all")

rawEisPlotFigure = plotNyquist(
    eisRaw,
    title="Raw EIS Data",
)
plt.legend().remove()

### Preprocessing

#### Erroneous Point Removal

Some plots contain points that are clearly erroneous and were tolerated due to the experiment time and temperature control constraints. In general, only single outliers in sequence are accepted (although there are some exceptions to this). This preprocessing step removes these points.

2 main erroneous modes were identified:

1. **50/100Hz Noise Issues**  
   Some spectra have outlying points at near 50Hz and 100Hz frequencies. This is likely due to an issue in the power supply (for which significant electronic effort in the Digatron design was made to mitigate). Without changing measurement channels (which would introduce other offsets), these points were tolerated.
2. **DSP Garbage In / Garbage Out**  
   Some batteries may have produced response signals that were not properly filtered or otherwise accepted by the Digatron DSP pipeline when computing impedance values. Often these are obvious due to negative real impedance values (which are impossible in reality).

The preprocessing step removes these points. Details of the removal are documented in the responsible functions.


In [33]:
from Lib.eisPreprocess import filterNegativeReal

eisNegRealProcessed = filterNegativeReal(eisRaw)

plotIndividualNyquist(
    eisNegRealProcessed,
    "../Plots/EIS/Processed/NegRealFilter/Main",
    False,
)
plt.close("all")

plotNyquist(
    eisNegRealProcessed,
    title="Negative Real Filter",
)
plt.legend().set_visible(False)
plt.show()

Filtered 4 negative real points from UCT_AST_A10_-20 RUN 2
Filtered 6 negative real points from UCT_AST_A10_-20 RUN 1
Filtered 3 negative real points from UCT_AST_B10_-20
Filtered 1 negative real points from UCT_AST_B09_+00
Filtered 1 negative real points from UCT_AST_B07_+RT2
Filtered 2 negative real points from UCT_AST_B05_+RT2
Filtered 1 negative real points from UCT_AST_B05_-40 3


/var/folders/3f/ct0r54cd0497vnq7h8jk96q40000gp/T/ipykernel_97866/1713288406.py:17: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [34]:
from Lib.eisPlot import plotNyquistComparison, EisDataComparison
from Lib.eisPreprocess import filterSingleOutlier


(eisSingleOutlierProcessed, singleOutlierModifiedSpectra) = filterSingleOutlier(
    eisNegRealProcessed
)

for spectra in singleOutlierModifiedSpectra:
    eisDataComparison = EisDataComparison(
        {
            "eis1": {
                "name": f"Pre-filter",
                "value": eisNegRealProcessed[spectra],
            },
            "eis2": {
                "name": f"Post-filter",
                "value": eisSingleOutlierProcessed[spectra],
            },
        }
    )

    plotNyquistComparison(
        eis1=eisDataComparison["eis1"],
        eis2=eisDataComparison["eis2"],
        mainTitle=f"Single Outlier Filter Comparison: {spectra}",
        figsize=(16, 9),
    )

del spectra

plotIndividualNyquist(
    eisSingleOutlierProcessed,
    "../Plots/EIS/Processed/SingleOutlierFilter/Main",
    False,
)
plt.close("all")

Filtered 1 outliers from UCT_AST_B07_-10 at frequencies [53.333]Hz
Filtered 1 outliers from UCT_AST_B05_-20 at frequencies [94.862]Hz
Filtered 1 outliers from UCT_AST_B09_+00 at frequencies [71.217]Hz


#### Renaming

While there was an attempt to standardize the naming of the files, there are still some inconsistencies. This step renames the files to a consistent format, given sufficient matching information in the current input.

Additionally, the dictionary is sorted by key name.


In [35]:
from Lib.eisPreprocess import renameLabels

eisRenamed = renameLabels(eisSingleOutlierProcessed)
eisRenamed = dict(sorted(eisRenamed.items()))

Rename: ➤ UCT_AST_9AH_A08_+RT       ➤ UCT_AST9AH_A08_RT1
Rename: ➤ UCT_AST_9AH_A05_+RT       ➤ UCT_AST9AH_A05_RT1
Rename: ➤ UCT_AST_9AH_A02_+RT       ➤ UCT_AST9AH_A02_RT1
Rename: ➤ UCT_AST_9AH_A07_+RT       ➤ UCT_AST9AH_A07_RT1
Rename: ➤ UCT_AST_9AH_A04_+RT       ➤ UCT_AST9AH_A04_RT1
Rename: ➤ UCT_AST_9AH_A03_+RT1      ➤ UCT_AST9AH_A03_RT1
Rename: ➤ UCT_AST_9AH_A09_+RT       ➤ UCT_AST9AH_A09_RT1
Rename: ➤ UCT_AST_9AH_A10_+RT       ➤ UCT_AST9AH_A10_RT1
Rename: ➤ UCT_AST_9AH_A06_+RT       ➤ UCT_AST9AH_A06_RT1
Rename: ➤ UCT_AST_9AH_A01_+RT       ➤ UCT_AST9AH_A01_RT1
Rename: ➤ UCT_AST_A04_-10           ➤ UCT_AST9AH_A04_-10
Rename: ➤ UCT_AST_A05_-10           ➤ UCT_AST9AH_A05_-10
Rename: ➤ UCT_AST_A10_-10           ➤ UCT_AST9AH_A10_-10
Rename: ➤ UCT_AST_A07_-10           ➤ UCT_AST9AH_A07_-10
Rename: ➤ UCT_AST_A06_-10           ➤ UCT_AST9AH_A06_-10
Rename: ➤ UCT_AST_A03_-10           ➤ UCT_AST9AH_A03_-10
Rename: ➤ UCT_AST_A08_-10           ➤ UCT_AST9AH_A08_-10
Rename: ➤ UCT_AST_A09_-10      

#### Named Metadata

The Batch, SoC and temperature metadata is extracted from the filenames and stored in the metadata. This is used for further analysis and plotting.

In [36]:
from Lib.eisImport import EisData
from Lib.eisPreprocess import disambiguateLabel, EisLabelDisambiguation
from Lib.eisAnalysis import getSocNumeric, getTemperatureNumeric


def addNameMetadata(name: str, eis: EisData) -> EisData:
    disambiguation: EisLabelDisambiguation = disambiguateLabel(name)

    eis.metadata["SoC"] = getSocNumeric(name)
    eis.metadata["SetTemperature"] = getTemperatureNumeric(name)
    eis.metadata["Batch"] = disambiguation.batteryBatch

    return eis


for name, data in eisRenamed.items():
    eisRenamed[name] = addNameMetadata(name, data)

del name, data

#### Handover


In [37]:
from copy import deepcopy

# This is the final processed EIS data. Treat as immutable from this point forward.
EIS = deepcopy(eisRenamed)


# Delete unused variables
del eisNegRealProcessed, eisSingleOutlierProcessed, eisRenamed

### Analysis

This section contains general analysis of the EIS data, including:

- **Constant Temperature, Varied SoC**  
  Analysis of the impedance spectra at a constant temperature and varied state of charge.
- **Constant SoC, Varied Temperature**  
  Analysis of the impedance spectra at a constant state of charge and varied temperature.
- **Batch A Before-After Room Temperature**  
  Analysis of the impedance spectra for Batch A before and after room temperature cycling.
- **Batch A vs Batch B Comparisons**  
  Analysis of the impedance spectra for Batch A and Batch B at the same state of charge and temperature.


#### Constant SoC, Varied Temp

In [38]:
import os
from concurrent import futures

from matplotlib.figure import Figure
from typing import Dict

from Lib.eisAnalysis import groupByBatteryNumber, groupByBatch, getSoC
from Lib.eisImport import EisData

eisByBattery = groupByBatteryNumber(EIS)


def plotConstSoCVariedTempNyquistBatchComparison(
    eisByBattery: Dict[str, Dict[str, EisData]],
    battery: str,
    saveDir: str,
) -> Figure:
    soc = getSoC(list(eisByBattery[battery].keys())[0])

    batchSplit = groupByBatch(eisByBattery[battery])

    fig, ax = plt.subplots(2, 1, figsize=(18, 24), sharey=True)

    fig.suptitle(
        f"{battery} ({soc}) - Constant SoC, Varied Temp",
        horizontalalignment="center",
    )

    for batch in batchSplit:
        plotNyquist(
            batchSplit[batch],
            title=f"Batch {batch}",
            ax=ax[0] if batch == "A" else ax[1],
        )

    ax[0].legend(loc="upper left", bbox_to_anchor=(1, 1))
    ax[1].legend(loc="upper left", bbox_to_anchor=(1, 1))

    ax[0].invert_yaxis()  # Shared y-axis nulled double inversion in plotNyquist
    (
        ax[0].set_xlim(ax[1].get_xlim())
        if ax[0].get_xlim() < ax[1].get_xlim()
        else ax[1].set_xlim(ax[0].get_xlim())
    )

    fig.savefig(
        f"{saveDir}/{battery}.png",
        transparent=True,
    )

    return fig


savePath = "../Plots/EIS/Analysis/ConstSocVariedTemp/Nyquist"
os.makedirs(savePath, exist_ok=True)
with futures.ThreadPoolExecutor() as executor:
    for battery in eisByBattery:
        executor.submit(
            plotConstSoCVariedTempNyquistBatchComparison,
            eisByBattery,
            battery,
            savePath,
        )

In [39]:
from Lib.eisPlot import plotBode


def plotConstantSocVariedTempBodePerBatch(
    eisByBattery: Dict[str, Dict[str, EisData]],
    battery: str,
    saveDir: str,
    **pltParams,
) -> Dict[str, Figure]:
    soc = getSoC(list(eisByBattery[battery].keys())[0])

    batchSplit = groupByBatch(eisByBattery[battery])

    os.makedirs(saveDir, exist_ok=True)

    batchFigures = {}

    for batch in batchSplit:
        fig = plotBode(
            batchSplit[batch],
            title=f"Batch {batch}, {battery} ({soc})",
            **pltParams,
        )

        batchFigures[batch] = fig

        fig.savefig(
            f"{saveDir}/{batch}{battery}.png",
            transparent=True,
            bbox_inches="tight",
        )

    return batchFigures


savePath = "../Plots/EIS/Analysis/ConstSocVariedTemp/Bode"
os.makedirs(savePath, exist_ok=True)
with futures.ThreadPoolExecutor() as executor:
    for battery in eisByBattery:
        executor.submit(
            plotConstantSocVariedTempBodePerBatch,
            eisByBattery,
            battery,
            savePath,
            xlim=(2, 10_000),
        )

#### Constant Temperature, Varied SoC

In [40]:
from Lib.eisAnalysis import groupByTemperature

eisByTemp = groupByTemperature(EIS)


def plotConstTempVariedSocNyquistBatchComparison(
    eisByTemp: Dict[str, Dict[str, EisData]],
    temp: str,
    saveDir: str | None = None,
    paperMode: bool = False,
) -> Figure | None:
    batchSplit = groupByBatch(eisByTemp[temp])

    fig, ax = plt.subplots(2, 1, figsize=(18, 24), sharey=True)

    if not paperMode:
        fig.suptitle(
            f"{temp}°C, Varied SoC",
            horizontalalignment="center",
        )

    for batch in batchSplit:
        plotNyquist(
            batchSplit[batch],
            title=f"Batch {batch} ({0 if batch == 'A' else 50} Cycles)",
            ax=ax[0] if batch == "A" else ax[1],
            paperMode=paperMode,
        )

    ax[0].legend(loc="upper left", bbox_to_anchor=(1, 1))
    ax[1].legend(loc="upper left", bbox_to_anchor=(1, 1)) if temp != "RT1" else None

    ax[0].invert_yaxis()  # Shared y-axis nulled double inversion in plotNyquist
    (
        ax[0].set_xlim(ax[1].get_xlim())
        if ax[0].get_xlim() < ax[1].get_xlim()
        else ax[1].set_xlim(ax[0].get_xlim())
    )

    if temp == "RT1":
        # Special case: RT1 bas no Batch B data
        ax[1].set_visible(False)
        ax[0].invert_yaxis()

    if saveDir:
        fig.savefig(
            f"{saveDir}/{temp}.png",
            transparent=True,
        )

    return fig


savePath = f"../Plots/EIS/Analysis/ConstTempVariedSoC"
os.makedirs(savePath, exist_ok=True)
with futures.ThreadPoolExecutor() as executor:
    for temp in eisByTemp:
        executor.submit(
            plotConstTempVariedSocNyquistBatchComparison,
            eisByTemp,
            temp,
            savePath,
        )

/var/folders/3f/ct0r54cd0497vnq7h8jk96q40000gp/T/ipykernel_97866/1648882891.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(2, 1, figsize=(18, 24), sharey=True)


In [41]:
from Lib.eisPlot import plotConstantTempVariedSocBodePerBatch

with futures.ThreadPoolExecutor() as executor:
    for temp in eisByTemp:
        executor.submit(
            plotConstantTempVariedSocBodePerBatch,
            eisByTemp,
            temp,
            f"../Plots/EIS/Analysis/ConstTempVariedSoC/Bode",
            xlim=(2, 10_000),
        )

#### Same State Comparisons

Compare all spectra of the same state of charge and temperature.

In [42]:
savePath = "../Plots/EIS/Analysis/SameStateComparison"


def splitEisSameState(
    eisByBattery: Dict[str, EisData]
) -> Dict[str, Dict[str, EisData]]:
    eisByBatteryByTemperature = {}
    for battery in eisByBattery:
        eisByBatteryByTemperature[battery] = groupByTemperature(eisByBattery[battery])
        # Merge RT1 and RT2
        eisByBatteryByTemperature[battery]["RT1"].update(
            eisByBatteryByTemperature[battery]["RT2"]
        )
        del eisByBatteryByTemperature[battery]["RT2"]
        eisByBatteryByTemperature[battery]["RT"] = eisByBatteryByTemperature[
            battery
        ].pop("RT1")

    return eisByBatteryByTemperature


eisSameState = splitEisSameState(eisByBattery)


def plotEisSameStateComparison(
    eisSameState: Dict[str, Dict[str, Dict[str, EisData]]],
):
    with futures.ThreadPoolExecutor() as executor:
        for battery in eisSameState:
            for temperature in eisSameState[battery]:
                if len(eisSameState[battery][temperature]) > 1:
                    soc = getSoC(list(eisSameState[battery][temperature].keys())[0])
                    executor.submit(
                        plotNyquist,
                        eisSameState[battery][temperature],
                        title=f"Same State Comparison Battery {battery} ({soc}) at {temperature}",
                        saveDir=savePath,
                        fileName=f"{battery}_{temperature}.png",
                        transparent=False,
                        limitFrequencyLabels=True,
                    )


_ = plotEisSameStateComparison(eisSameState)

### High Frequency Resistance

The high frequency resistance is an equivalent circuit element of interest.

It may be read as the simple minima resistance within the spectra.

In [43]:
from tabulate import tabulate

output = []
rHighFrequency = {}

for battery in eisByBattery:

    for spectra in eisByBattery[battery]:
        disambiguation = disambiguateLabel(spectra)

        zreal1 = eisByBattery[battery][spectra].data["Zreal1"]
        zreal2 = eisByBattery[battery][spectra].data["Zreal2"]
        zreal3 = eisByBattery[battery][spectra].data["Zreal3"]

        meanZreal = (zreal1 + zreal2 + zreal3) / 3
        minIndex = meanZreal.idxmin()
        minZreal = meanZreal[minIndex]
        stdMinZreal = (
            (
                (zreal1[minIndex] - minZreal) ** 2
                + (zreal2[minIndex] - minZreal) ** 2
                + (zreal3[minIndex] - minZreal) ** 2
            )
            / 3
        ) ** 0.5

        output.append(
            [
                f"{disambiguation.batteryBatch}{battery}",
                f"{disambiguation.temperature}",
                spectra,
                f"{minZreal:.2f}",
                f"{stdMinZreal:.3f} mΩ",
            ]
        )

        rHighFrequency[
            (
                f"{disambiguation.batteryBatch}{battery}",
                f"{disambiguation.temperature}",
            )
        ] = (minZreal, stdMinZreal)


print(
    tabulate(
        output,
        headers=["Battery", "Temperature", "Spectra", "Min Zreal", "± Std Deviation"],
        floatfmt=("", "", "", ".2f", ".3f"),
    )
)

Battery    Temperature    Spectra                 Min Zreal  ± Std Deviation
---------  -------------  --------------------  -----------  -----------------
A01        -10            UCT_AST9AH_A01_-10          18.86  0.043 mΩ
A01        -20            UCT_AST9AH_A01_-20          24.46  0.002 mΩ
A01        -30            UCT_AST9AH_A01_-30          33.03  0.007 mΩ
A01        -40            UCT_AST9AH_A01_-40          37.60  0.003 mΩ
A01        00             UCT_AST9AH_A01_00           18.59  0.021 mΩ
A01        RT1            UCT_AST9AH_A01_RT1          17.03  0.009 mΩ
A01        RT2            UCT_AST9AH_A01_RT2          27.19  0.009 mΩ
B01        -10            UCT_AST9AH_B01_-10          30.43  0.004 mΩ
B01        -20            UCT_AST9AH_B01_-20          35.47  0.005 mΩ
B01        -30            UCT_AST9AH_B01_-30          30.99  0.015 mΩ
B01        -40            UCT_AST9AH_B01_-40          44.05  0.017 mΩ
B01        00             UCT_AST9AH_B01_00           39.60  0.007 mΩ
B01 

In [44]:
# Mean Cell Mass Precents and Freezing Points, simply manually imported here (finalised).
cellData = {
    "A01": {
        "A": 0.424,
    },
    "A02": {
        "A": 0.410,
    },
    "A03": {
        "A": 0.385,
    },
    "A04": {
        "A": 0.372,
    },
    "A05": {
        "A": 0.334,
    },
    "A06": {
        "A": 0.318,
    },
    "B01": {
        "A": 0.372,
    },
    "B02": {
        "A": 0.397,
    },
    "B03": {
        "A": 0.357,
    },
    "B04": {
        "A": 0.333,
    },
    "B05": {
        "A": 0.311,
    },
    "B06": {
        "A": 0.284,
        "Tfus": -30.80,
    },
    "A07": {
        "A": 0.282,
        "Tfus": -30.20,
    },
    "B07": {
        "A": 0.249,
        "Tfus": -21.74,
    },
    "A08": {
        "A": 0.265,
        "Tfus": -25.58,
    },
    "B08": {
        "A": 0.220,
        "Tfus": -16.43,
    },
    "A09": {
        "A": 0.222,
        "Tfus": -16.73,
    },
    "B09": {
        "A": 0.156,
        "Tfus": -8.87,
    },
    "A10": {
        "A": 0.180,
        "Tfus": -11.26,
    },
    "B10": {
        "A": 0.149,
        "Tfus": -8.28,
    },
}

# Thermal equilibrium Mass Percentages for EIS temperature set points
eqData = {
    "-40": {
        "A_E": 0.3089,
    },
    "-30": {
        "A_E": 0.2811,
    },
    "-20": {
        "A_E": 0.2407,
    },
    "-10": {
        "A_E": 0.1678,
    },
}

# Calculate the Freezing Factor for each battery

fCorrelation = []
for battery in cellData:
    for setPoint in eqData:
        t = int(setPoint)

        if cellData[battery].get("Tfus", None) is None:
            continue

        if t > cellData[battery]["Tfus"]:
            continue

        f = 1 - cellData[battery]["A"] / eqData[setPoint]["A_E"]
        rHf = rHighFrequency.get((battery, setPoint), None)
        if rHf is None:
            continue

        print(f"{battery} at {t}°C: {f:.3f} (Rhf: {rHf[0]:.2f} ± {rHf[1]:.3f} mΩ)")

        # Drop A10 and B10 -20°C, deemed outliers
        if (battery == "A10" or battery == "B10") and t == -20:
            continue

        fCorrelation.append((f, rHf))

B06 at -40°C: 0.081 (Rhf: 63.51 ± 0.017 mΩ)
A07 at -40°C: 0.087 (Rhf: 50.84 ± 0.003 mΩ)
B07 at -40°C: 0.194 (Rhf: 92.70 ± 0.013 mΩ)
B07 at -30°C: 0.114 (Rhf: 87.02 ± 0.011 mΩ)
A08 at -30°C: 0.057 (Rhf: 58.49 ± 0.005 mΩ)
B08 at -40°C: 0.288 (Rhf: 151.90 ± 0.017 mΩ)
B08 at -30°C: 0.217 (Rhf: 136.26 ± 0.013 mΩ)
B08 at -20°C: 0.086 (Rhf: 71.14 ± 0.003 mΩ)
A09 at -30°C: 0.210 (Rhf: 154.64 ± 0.026 mΩ)
A09 at -20°C: 0.078 (Rhf: 110.26 ± 0.011 mΩ)
B09 at -20°C: 0.352 (Rhf: 188.37 ± 0.014 mΩ)
B09 at -10°C: 0.070 (Rhf: 47.70 ± 0.003 mΩ)
A10 at -20°C: 0.252 (Rhf: 354.94 ± 0.248 mΩ)
B10 at -20°C: 0.381 (Rhf: 372.64 ± 0.039 mΩ)
B10 at -10°C: 0.112 (Rhf: 107.25 ± 0.006 mΩ)


In [45]:
from Lib.eisPlot import plotHighFrequencyResistanceVsFreezingFactor

_ = plotHighFrequencyResistanceVsFreezingFactor(fCorrelation)

#### DC Voltage

In [46]:
from Lib.eisPlot import plotDcVoltageByBattery, plotDcVoltageByTemperature

# B05 EIS tests we re-run at -40 and -20 degrees Celsius multiple times, invalidating these results.
# Filter these for DC voltage analysis.
EIS_DC = {
    key: value
    for key, value in EIS.items()
    if not ("B05" in key and ("-40" in key or "-20" in key))
}

dcVoltageSaveDir = "../Plots/Voltage"
os.makedirs(dcVoltageSaveDir, exist_ok=True)
dcVoltageOverSocFig = plotDcVoltageByBattery(
    EIS_DC,
    cellData=cellData,
    savePath=f"{dcVoltageSaveDir}/DCVoltageOverSoC.png",
)

In [47]:
dcVoltageOverTemperatureFig = plotDcVoltageByTemperature(
    EIS_DC,
    savePath=f"{dcVoltageSaveDir}/DCVoltageOverSoCByTemperature.png",
)

## Export

Export files intended for use in latex manuscripts.

In [48]:
import importlib

matplotlib.use("pgf")

importlib.reload(plt)

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "text.usetex": True,
        "pgf.rcfonts": False,
        # "pgf.preamble": r"\DeclareUnicodeCharacter{03A9}{\Omega}",
    }
)

latexSaveDir = "../Plots/Latex"
os.makedirs(latexSaveDir, exist_ok=True)

In [49]:
temperatureRangeFig = plotEisTestTemperatureRanges(
    eisObservations,
    paperMode=True,
)

temperatureRangeFig.savefig(
    os.path.join(latexSaveDir, "EIS_TemperatureRanges.pgf"),
    bbox_inches="tight",
)

In [50]:
for temp in ["-40", "-30", "-20", "-10", "00", "RT1", "RT2"]:
    eisTempFigure = plotConstTempVariedSocNyquistBatchComparison(
        eisByTemp,
        temp,
        paperMode=True,
    )
    figureName = temp if temp[:2] == "RT" else f"M{temp[-2:]}"
    eisTempFigure.savefig(
        os.path.join(latexSaveDir, f"EIS_{figureName}.pgf"),
        bbox_inches="tight",
    )

In [51]:
emfByBatteryFigure = plotDcVoltageByBattery(
    EIS_DC,
    cellData=cellData,
    paperMode=True,
)
# Remove title
emfByBatteryFigure.suptitle("")
emfByBatteryFigure.savefig(
    os.path.join(latexSaveDir, "EMF_Battery.pgf"),
    bbox_inches="tight",
)

In [52]:
freezingFactorCorrelationFigure = plotHighFrequencyResistanceVsFreezingFactor(
    fCorrelation
)
freezingFactorCorrelationFigure.savefig(
    os.path.join(latexSaveDir, "FreezingExtentCorrelation.pgf"),
    bbox_inches="tight",
)